In [13]:
import numpy as np
import pandas as pd
import torch
from PIL import Image
from torchvision import transforms
import os
import matplotlib.pyplot as plt
import ast
from functools import partial

Use WordNet similarity score to find probabilitiy corresponding to the closest label 

In [14]:
from nltk.corpus import wordnet

In [35]:
df = pd.read_csv('alexnet/alexnet_scores_wordnet_id.csv',index_col=0)

Input dataframe columns: (see alexnet.ipynb for getting wordnet ids for corresponding labels)

In [36]:
df.columns

Index(['type', 'dir', 'id_labels', 'readable_labels'], dtype='object')

In [17]:
# manually select the synsnet with the correct definition for each type
type_names = np.unique(df['type'].values)
for t in type_names:
    syns = wordnet.synsets(t.lower())
    print(f'{t}: ')
    for s in syns:
        print(s)
        print(s.definition())
    print()

Banana: 
Synset('banana.n.01')
any of several tropical and subtropical treelike herbs of the genus Musa having a terminal crown of large entire leaves and usually bearing hanging clusters of elongated fruits
Synset('banana.n.02')
elongated crescent-shaped yellow fruit with soft sweet flesh

Beach: 
Synset('beach.n.01')
an area of sand sloping down to the water of a sea or lake
Synset('beach.v.01')
land on a beach

Car: 
Synset('car.n.01')
a motor vehicle with four wheels; usually propelled by an internal combustion engine
Synset('car.n.02')
a wheeled vehicle adapted to the rails of railroad
Synset('car.n.03')
the compartment that is suspended from an airship and that carries personnel and the cargo and the power plant
Synset('car.n.04')
where passengers ride up and down
Synset('cable_car.n.01')
a conveyance for passengers or freight on a cable railway

Church: 
Synset('church.n.01')
one of the groups of Christians who have their own beliefs and forms of worship
Synset('church.n.02')
a 

In [18]:
type_synsets = ['banana.n.02','beach.n.01','car.n.01','church.n.02','beacon.n.03','mountain.n.01','coffee_mug.n.01','airplane.n.01']
type_final = {type_names[i] : type_synsets[i] for i in range(len(type_names))}

In [19]:
type_final

{'Banana': 'banana.n.02',
 'Beach': 'beach.n.01',
 'Car': 'car.n.01',
 'Church': 'church.n.02',
 'Lighthouse': 'beacon.n.03',
 'Mountain': 'mountain.n.01',
 'Mug': 'coffee_mug.n.01',
 'Plane': 'airplane.n.01'}

In [20]:
# compute similarity score between synsets of each of the top 10 labels and the type 
def getSyns(similarity,closest,row):
    # type name
    category = row['type'] 
    
    # top 10 labels' ids in wordnet
    labels  = row['id_labels'] 
    labels = ast.literal_eval(labels)
    key = labels.keys() 
    
    # iterate through top 10 labels and compare with type 
    sim = dict()
    notFound = True
    for k in key:
        pos, synid = k.split('-')[1], int(k.split('-')[0]) # part of speech (all noun in this case), id
        k_syns = wordnet.synset_from_pos_and_offset(pos, synid) # label synsets
        cat_syns = wordnet.synset(type_final[category]) # type synsets
        s = k_syns.path_similarity(cat_syns)
        sim[k] = s
        
        # find the first > 0.5 similarity label among the top 10
        if s >= 0.5 and notFound:
            closest.append({k:s})
            notFound = False
            
    # return the label with highest probabilitiy if none is above 0.5
    if notFound:
        sim_np = np.array(sim.values())
        max_key = list(key)[np.argmax(sim_np)]
        closest.append({max_key : sim[max_key]})
    
    similarity.append(sim)

In [23]:
# read labels to wordnet synsets
ltw = pd.read_json('imagenet_label_to_wordnet_synset.json').T

In [24]:
# get readable labels from imagenet_label_to_wordnet_synset.json
def convertIdToReadable(id_label):
    for idx in range(ltw.shape[0]):
        if ltw['id'][idx] == id_label:
            return ltw['label'][idx]
    return None

In [25]:
similarity = []
closest = []
getSynsPartial = partial(getSyns, similarity, closest)
df.apply(getSynsPartial, axis=1)

df['similarity_score'] = similarity
df['closest_id_label'] = [list(item.keys())[0] for item in closest]
df['closest_similarity_score'] = [list(item.values())[0] for item in closest]
df['closest_readable_label'] = df['closest_id_label'].apply(convertIdToReadable)
df['closest_prob'] = [ast.literal_eval(d)[i] for d, i in zip(df['id_labels'],df['closest_id_label'])]

In [26]:
df

,type,dir,id_labels,readable_labels,similarity_score,closest_id_label,closest_similarity_score,closest_readable_label,closest_prob
0,Mountain,Image/All_Cropped\Mountain\1.jpg,"{'09193705-n': 0.344296395778656, '09332890-n'...","{'alp': 0.344296395778656, 'lakeside': 0.09276...","{'09193705-n': 0.5, '09332890-n': 0.2, '094686...",09193705-n,0.5,alp,0.344296
1,Mountain,Image/All_Cropped\Mountain\10.jpg,"{'09468604-n': 0.34921252727508545, '09193705-...","{'valley': 0.34921252727508545, 'alp': 0.25682...","{'09468604-n': 0.2, '09193705-n': 0.5, '094725...",09193705-n,0.5,alp,0.256830
2,Mountain,Image/All_Cropped\Mountain\11.jpg,"{'09472597-n': 0.7669864892959595, '09193705-n...","{'volcano': 0.7669864892959595, 'alp': 0.10891...","{'09472597-n': 0.5, '09193705-n': 0.5, '094686...",09472597-n,0.5,volcano,0.766986
3,Mountain,Image/All_Cropped\Mountain\12.jpg,"{'09468604-n': 0.48895254731178284, '09193705-...","{'valley': 0.48895254731178284, 'alp': 0.41202...","{'09468604-n': 0.2, '09193705-n': 0.5, '037929...",09193705-n,0.5,alp,0.412028
4,Mountain,Image/All_Cropped\Mountain\13.jpg,"{'09246464-n': 0.7365226149559021, '09399592-n...","{'cliff': 0.7365226149559021, 'promontory': 0....","{'09246464-n': 0.25, '09399592-n': 0.333333333...",09193705-n,0.5,alp,0.013008
...,...,...,...,...,...,...,...,...,...
122,Church,Image/All_Cropped\Church\5.jpg,"{'03028079-n': 0.8215102553367615, '02825657-n...","{'church': 0.8215102553367615, 'bell cote': 0....","{'03028079-n': 1.0, '02825657-n': 0.1111111111...",03028079-n,1.0,"church, church building",0.821510
123,Church,Image/All_Cropped\Church\6.jpg,"{'02793495-n': 0.9915186762809753, '02859443-n...","{'barn': 0.9915186762809753, 'boathouse': 0.00...","{'02793495-n': 0.2, '02859443-n': 0.1666666666...",03028079-n,1.0,"church, church building",0.001823
124,Church,Image/All_Cropped\Church\7.jpg,"{'03028079-n': 0.38567203283309937, '03781244-...","{'church': 0.38567203283309937, 'monastery': 0...","{'03028079-n': 1.0, '03781244-n': 0.1428571428...",03028079-n,1.0,"church, church building",0.385672
125,Church,Image/All_Cropped\Church\8.jpg,"{'03028079-n': 0.4912656247615814, '02793495-n...","{'church': 0.4912656247615814, 'barn': 0.25147...","{'03028079-n': 1.0, '02793495-n': 0.2, '039303...",03028079-n,1.0,"church, church building",0.491266


Potential issues: 
1. top 10 and threshold of 0.5 are arbitrarily chosen - e.g. see `df.loc[4]` where the top label is `cliff` (similarity score 0.25) but `alp` is the chosen label
2. when none of top 10 labels' similarity scores pass the 0.5 threshold, it chooses the label with max similarity score and breaks ties by choosing the first, which is one with highest probability among all max similarity score labels - e.g. see `df.loc[9]`


In [34]:
df.loc[4]

type                                                                 Mountain
dir                                         Image/All_Cropped\Mountain\13.jpg
id_labels                   {'09246464-n': 0.7365226149559021, '09399592-n...
readable_labels             {'cliff': 0.7365226149559021, 'promontory': 0....
similarity_score            {'09246464-n': 0.25, '09399592-n': 0.333333333...
closest_id_label                                                   09193705-n
closest_similarity_score                                                  0.5
closest_readable_label                                                    alp
closest_prob                                                        0.0130077
Name: 4, dtype: object

In [33]:
df.loc[9]

type                                                                 Mountain
dir                                          Image/All_Cropped\Mountain\3.jpg
id_labels                   {'09288635-n': 0.13967332243919373, '09332890-...
readable_labels             {'geyser': 0.13967332243919373, 'lakeside': 0....
similarity_score            {'09288635-n': 0.2, '09332890-n': 0.2, '029513...
closest_id_label                                                   09288635-n
closest_similarity_score                                                  0.2
closest_readable_label                                                 geyser
closest_prob                                                         0.139673
Name: 9, dtype: object

In [219]:
# df.to_csv('alexnet_scores_wordnet_id_closest_label.csv')